In [13]:
# 필요한 라이브러리 설치
# transformers: 다양한 사전 학습된 모델을 제공하는 라이브러리
# sentencepiece: 텍스트 토크나이징을 위한 라이브러리
# onnxruntime: ONNX 모델을 실행하기 위한 런타임
# gradio: 웹 인터페이스를 쉽게 만들 수 있는 라이브러리
!pip install -q transformers sentencepiece onnxruntime gradio

# onnx 패키지 설치
# ONNX(Open Neural Network Exchange) 모델을 다루기 위한 패키지
!pip install onnx

# 필요한 모듈 임포트
import re  # 정규 표현식 모듈
import torch  # PyTorch 라이브러리
import onnxruntime as ort  # ONNX 런타임
from transformers import AutoTokenizer, AutoModelForSequenceClassification  # 트랜스포머 모델과 토크나이저
import numpy as np  # 수치 계산 라이브러리
import pandas as pd  # 데이터프레임 처리 라이브러리
import gradio as gr  # 웹 인터페이스 라이브러리
from openai import OpenAI  # OpenAI API 사용을 위한 라이브러리

In [14]:
# KoBERT 모델과 토크나이저 로드
# "monologg/kobert" 사전 학습된 KoBERT 모델과 해당 토크나이저를 로드
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
model = AutoModelForSequenceClassification.from_pretrained("monologg/kobert", num_labels=3)

# ONNX 모델 저장 경로 설정
onnx_model_path = "kobert_sentiment.onnx"

# PyTorch 모델을 ONNX 형식으로 변환 및 저장
torch.onnx.export(
    model,  # 변환할 모델
    (torch.tensor([[1]]), torch.tensor([[1]])),  # 더미 입력 (input_ids, attention_mask)
    onnx_model_path,  # 저장할 파일 경로
    input_names=["input_ids", "attention_mask"],  # 입력 이름 지정
    output_names=["logits"],  # 출력 이름 지정
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "sequence_length"},  # 동적 배치 크기 및 시퀀스 길이
        "attention_mask": {0: "batch_size", 1: "sequence_length"},
        "logits": {0: "batch_size"}
    },
    opset_version=14  # ONNX opset 버전 지정
)

# ONNX 모델 로드
# 변환된 ONNX 모델을 InferenceSession으로 로드하여 추론 준비
ort_session = ort.InferenceSession(onnx_model_path)

The repository for monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] ㅛ
The repository for monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
def predict_sentiment_onnx(text):
    """
    주어진 텍스트의 감정을 예측하는 함수.

    Args:
        text (str): 분석할 텍스트.

    Returns:
        tuple: 예측된 레이블(정수)과 해당 레이블의 신뢰도.
    """
    # 텍스트를 토크나이즈하여 NumPy 배열 형태로 변환
    inputs = tokenizer(text, return_tensors="np", padding=True, truncation=True, max_length=128)
    ort_inputs = {
        "input_ids": inputs["input_ids"],  # 입력 토큰 IDs
        "attention_mask": inputs["attention_mask"]  # 어텐션 마스크
    }
    # ONNX 모델을 사용하여 로짓(logits) 계산
    logits = ort_session.run(None, ort_inputs)[0]
    # 소프트맥스를 적용하여 확률로 변환
    probabilities = torch.nn.functional.softmax(torch.from_numpy(logits), dim=-1).detach().numpy()
    # 가장 높은 확률을 가진 레이블 선택
    predicted_label = np.argmax(probabilities, axis=1)[0]
    # 해당 레이블의 신뢰도 추출
    confidence = probabilities[0][predicted_label]
    return predicted_label, confidence

In [16]:
def calculate_affinity_score(messages):
    """
    메시지 리스트를 기반으로 호감도 점수를 계산하는 함수.

    Args:
        messages (list): 각 메시지에 대한 정보(감정 및 신뢰도)를 포함한 딕셔너리 리스트.

    Returns:
        float: 0부터 100 사이의 호감도 점수.
    """
    score = 0  # 초기 점수
    # 감정에 따른 가중치 설정
    sentiment_weights = {
        "긍정": 1.0,
        "중립": 0.0,
        "부정": -1.0
    }
    # 각 메시지의 감정에 따라 점수 계산
    for msg in messages:
        weight = sentiment_weights.get(msg["sentiment"], 0)  # 감정에 따른 가중치
        score += weight * msg["confidence"]  # 가중치와 신뢰도를 곱하여 점수에 추가
    # 전체 점수를 0에서 100 사이로 정규화
    total_score = (score / len(messages)) * 50 + 50
    return max(0, min(total_score, 100))  # 점수가 0보다 작거나 100보다 크지 않도록 제한

In [17]:
def analyze_conversation(file_path, api_key):
    """
    대화 파일을 분석하여 요약, 호감도 점수, 평가, 조언을 제공하는 함수.

    Args:
        file_path (str): 분석할 대화 파일의 경로 (CSV 형식).
        api_key (str): Upstage API 키.

    Returns:
        tuple: 대화 요약, 호감도 점수, 호감도 평가, 조언.
    """
    # 메시지 전처리
    # CSV 파일을 읽어와 메시지 리스트 생성
    df = pd.read_csv(file_path, delimiter=",", encoding='utf-8')
    messages = [{"time": row['Date'], "sender": row['User'], "message": row['Message']} for _, row in df.iterrows()]

    # 감정 분석
    sentiment_labels = {0: "부정", 1: "중립", 2: "긍정"}  # 레이블과 감정 매핑
    for msg in messages:
        label, confidence = predict_sentiment_onnx(msg["message"])  # 감정 예측
        msg["sentiment"] = sentiment_labels.get(label, "알 수 없음")  # 감정 레이블 추가
        msg["confidence"] = confidence  # 신뢰도 추가

    # 호감도 점수 계산
    affinity_score = calculate_affinity_score(messages)

    # 호감도 평가
    if 0 <= affinity_score <= 20:
        evaluation = "강한 부정"
    elif 21 <= affinity_score <= 40:
        evaluation = "약한 부정"
    elif 41 <= affinity_score <= 60:
        evaluation = "중립"
    elif 61 <= affinity_score <= 80:
        evaluation = "약한 긍정"
    elif 81 <= affinity_score <= 100:
        evaluation = "강한 긍정"

    # Upstage 솔라 API 설정
    # OpenAI API 클라이언트를 Upstage 솔라 엔드포인트로 초기화
    client = OpenAI(api_key=api_key, base_url="https://api.upstage.ai/v1/solar")

    # 대화 요약
    # 모든 메시지를 하나의 텍스트로 결합
    conversation_text = "\n".join([f"{msg['sender']}: {msg['message']}" for msg in messages])
    summary_prompt = (
        "아래는 두 사람 간의 대화 내용입니다. 이 대화를 요약하여 주요 주제, 상대방의 관심사, 감정 등을 파악해주세요:\n\n"
        f"{conversation_text}\n\n요약:"
    )
    # 요약 생성 요청
    summary_result = client.chat.completions.create(
        model="solar-1-mini-chat",
        messages=[
            {"role": "system", "content": "당신은 전문 요약가입니다."},
            {"role": "user", "content": summary_prompt}
        ]
    )
    # 요약된 내용 추출
    conversation_summary = summary_result.choices[0].message.content.strip()

    # 조언 생성
    prompt = (
        f"아래는 두 사람 간의 대화 요약입니다:\n\n{conversation_summary}\n\n"
        f"대화 상대의 호감도 점수가 {affinity_score:.2f}점이며, 평가 결과는 '{evaluation}'입니다. "
        f"이 요약된 대화를 바탕으로 상대방의 성향을 분석하고, 그에 따른 맞춤형 조언을 제공해주세요."
    )
    # 조언 생성 요청
    advice_result = client.chat.completions.create(
        model="solar-1-mini-chat",
        messages=[
            {"role": "system", "content": "당신은 연애 상담 전문가입니다."},
            {"role": "user", "content": prompt}
        ]
    )
    # 조언 내용 추출
    advice = advice_result.choices[0].message.content.strip()

    return conversation_summary, f"{affinity_score:.2f}", evaluation, advice

In [18]:
# Gradio 인터페이스 정의
# 사용자로부터 입력을 받고 결과를 출력하는 웹 인터페이스 구성
gr.Interface(
    fn=analyze_conversation,  # 호출할 함수
    inputs=[
        gr.File(label="카카오톡 채팅 파일 (CSV)"),  # CSV 파일 업로드 입력
        gr.Textbox(label="Upstage API Key", type="password")  # API 키 입력 (비밀번호 형태)
    ],
    outputs=[
        gr.Textbox(label="대화 요약"),  # 대화 요약 출력
        gr.Textbox(label="호감도 점수"),  # 호감도 점수 출력
        gr.Textbox(label="호감도 평가"),  # 호감도 평가 출력
        gr.Textbox(label="조언")  # 조언 출력
    ],
    title="카카오톡 대화 분석 및 호감도 평가"  # 인터페이스 제목
).launch()  # 웹 인터페이스 실행

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2af337d2084cf3c489.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
